In [ ]:
import cell_movie_maker as cmm
import chaste_simulation_database_connector as csdc
import matplotlib.pyplot as plt
import numpy as np
import pathlib


from IPython.display import display
import IPython.display


cmm.Config.simulations_folder = pathlib.Path("chaste_output").resolve()
cmm.Config.output_folder = pathlib.Path("analysis_output").resolve()
cmm.Config.set_simulation_database(pathlib.Path("analysis_output", "TCellABM.db"))

experiment = cmm.Experiment(pathlib.Path("chaste_output", "TCellABM"))
simulation = experiment.simulations[0]
sample_timepoint = simulation.timepoints[50]


Most of this requires chaste-simulation-database-connector module

In [ ]:
import chaste_simulation_database_connector as csdc

Retrieving data from the database is setup using API like infrastructure.  
Retrieval works by configuring a `Collector` object with what data should be collected.  
The configured `Collector` can then be asked to perform the collection with optional filters e.g. which experiment to retrieve data for.  

Complex construction of `Collector` classes can be a weakness of this design pattern.  
To abstract away the construction of `Collector` classes, we make use of a Factory pattern.  
All the retrieval example presented below a structured as:  
`Database.CollectorFactory(AbstractCollector).collect(experiment)`

The advantage of this method is that it is designed with usability in mind, and it is intended that the line above be interpreted as:  
Database.I want to retrieve something(This is what I want to retrieve).collect(for this experiment)




# Read parameters
By default, when parameters are written to the database a was_varied flag automatically set depending on whether the parameter varies across the experiment.  
In this demo, since there is only 1 parameter in the experiment we have to specify to retrieve all parameters, not just parameters that were varied.  

In [10]:
parameters = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.ParametersCollector).collect(experiment, only_varied=False)
display(parameters)

parameter_name,ACCD_S,ACCD_T,AngiogenesisDelay,AngiogenesisRadius,CCL5CD8Sensitivity,CCL5ConsumptionCoefficient,CCL5DecayRate,CCL5DiffusionCoefficient,CCL5DuDtCoefficient,CCL5ExtravasationThreshold,...,TumourNecroticConcentration,TumourQuiescenceVolume,VesselHypoxicThreshold,VesselMaxRadius,VesselMinRadius,VesselOccludedDeathDelay,VesselOcclusionThreshold,VesselPressure,VesselRadiusDamping,WIDTH
iteration,,,,,,,,,,,,,,,,,,,,,
1520,32,24,12,0,5,0.03,0.01,1,1,0.1,...,0.01,0,0.1,0.75,0,6,0.75,-1,10,100


# Read Analysis
There are other ways of retrieving analysis (e.g. by analysis name), see source code for details.  
These examples show how to retrieve analysis using provide Collector implementations.

## Check What Analysis is Stored in the Database

In [13]:
collector = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.AnalysisCollector)
collector.collect_analysis_names()

,analysis_name
0,Roundness alpha=0.50
1,TCell-TCell PCF r=50 dr=1 step=1
2,Tumour Region Sizes alpha=0.50
3,Tumour Region TCell Counts alpha=0.50 buffer_s...
4,Tumour-TCell PCF r=50 dr=1 step=1
5,Tumour-Tumour PCF r=50 dr=1 step=1
6,Tumour-Vessel PCF r=50 dr=1 step=1
7,Vessel-TCell PCF r=50 dr=1 step=1
8,cellcounts


## Cellcounts (from analysis performed by `Preprocessor`)

In [37]:
collector = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.CellcountCollector)
analysis = collector.collect(experiment)
display(analysis.head(10))

100%|██████████| 1/1 [00:00<00:00, 46.76it/s]


n_tumour  n_tumour_hypoxic  n_tumour_necrotic  \
iteration timestep                                                  
1520      0                4                 0                  0   
          600              5                 0                  0   
          1200             5                 0                  0   
          1800             6                 0                  0   
          2400             7                 0                  0   
          3000             9                 0                  0   
          3600            10                 0                  0   
          4200            10                 0                  0   
          4800            12                 0                  0   
          5400            16                 0                  0   

                    mean_tumour_oxygen  median_tumour_oxygen  \
iteration timestep                                             
1520      0                   0.999735              0.999728   
          600                 0.970322              0.969931   
          1200                0.955396              0.955028   
          1800                0.942456              0.942593   
          2400                0.927253              0.928161   
          3000                0.912096              0.912162   
          3600                0.897399              0.897408   
          4200                0.884108              0.883923   
          4800                0.870021              0.870445   
          5400                0.853261              0.851947   

                    mean_tumour_radius  median_tumour_radius  n_tcells  
iteration timestep                                                      
1520      0                   0.392259              0.392259         0  
          600                 0.484881              0.487153         0  
          1200                0.484881              0.487153         0  
          1800                0.479744              0.487207         0  
          2400                0.480557              0.487193         0  
          3000                0.474991              0.481037         0  
          3600                0.472470              0.475706         0  
          4200                0.472478              0.475724         0  
          4800                0.471173              0.478548         0  
          5400                0.465780              0.469153         0

## Roundness

In [36]:
collector = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.RoundnessCollector)
analysis = collector.collect(experiment)
display(analysis.head(10))

100%|██████████| 1/1 [00:00<00:00, 75.94it/s]


roundness
iteration timestep           
1520      60000      0.647359
          59400      0.118653
          58800      0.040252
          58200      0.025326
          57600      0.032809
          57000      0.033988
          56400      0.627372
          55800      0.642522
          55200      0.628499
          54600      0.728608

## Tumour Region Sizes

In [35]:
collector = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.TumourRegionSizeCollector)
analysis = collector.collect(experiment)
display(analysis.head(10))

100%|██████████| 1/1 [00:00<00:00, 45.36it/s]


tumour_area  normoxic_area  hypoxic_area  necrotic_area
iteration timestep                                                         
1520      60000       12.695242      12.695242      0.000000              0
          59400       30.484600      30.484600      0.000000              0
          58800       97.044701      97.044701      0.000000              0
          58200      367.563111     367.513986      0.049125              0
          57600      698.547497     698.522758      0.024739              0
          57000     1076.616890    1076.423196      0.193695              0
          56400     1628.962152    1626.012488      2.949665              0
          55800     1927.781439    1918.074306      9.707132              0
          55200     2199.268527    2179.044203     20.224324              0
          54600     2407.224481    2345.276193     61.948288              0

## Tumour Region T-Cell Counts

In [34]:
collector = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.TumourRegionTCellCountCollector)
analysis = collector.collect(experiment)
display(analysis.head(10))

100%|██████████| 1/1 [00:00<00:00, 45.96it/s]


necrotic_region_count  necrotic_region_density  \
iteration timestep                                                   
1520      60000                         0                        0   
          59400                         0                        0   
          58800                         0                        0   
          58200                         0                        0   
          57600                         0                        0   
          57000                         0                        0   
          56400                         0                        0   
          55800                         0                        0   
          55200                         0                        0   
          54600                         0                        0   

                    hypoxic_region_count  hypoxic_region_density  \
iteration timestep                                                 
1520      60000                        0                0.000000   
          59400                        0                0.000000   
          58800                        0                0.000000   
          58200                        0                0.000000   
          57600                        0                0.000000   
          57000                        0                0.000000   
          56400                        2                0.678043   
          55800                        4                0.412068   
          55200                        6                0.296672   
          54600                       29                0.468132   

                    normoxic_region_count  normoxic_region_density  \
iteration timestep                                                   
1520      60000                         8                 0.630157   
          59400                        21                 0.688872   
          58800                        47                 0.484313   
          58200                       166                 0.451683   
          57600                       331                 0.473857   
          57000                       530                 0.492371   
          56400                       811                 0.498766   
          55800                       977                 0.509365   
          55200                      1147                 0.526378   
          54600                      1250                 0.532986   

                    buffer_region_count  buffer_region_density  \
iteration timestep                                               
1520      60000                      71               0.452705   
          59400                     317               0.477166   
          58800                     752               0.472244   
          58200                    1186               0.450234   
          57600                    1122               0.433301   
          57000                    1233               0.442705   
          56400                    1081               0.423885   
          55800                    1031               0.416108   
          55200                     913               0.392183   
          54600                     805               0.364733   

                    exterior_region_count  exterior_region_density  
iteration timestep                                                  
1520      60000                      3405                 0.346372  
          59400                      3059                 0.328742  
          58800                      2509                 0.301905  
          58200                      1877                 0.268210  
          57600                      1694                 0.252383  
          57000                      1312                 0.213742  
          56400                      1113                 0.191210  
          55800                       914                 0.163375  
          55200                       

## PCFs

In [18]:
pcf_collector = cmm.Config.simulation_database.CollectorFactory(csdc.collectors.PCFCollector)

In [29]:
print("Tumour-Vessel")
analyser = cmm.analysers.TumourVesselPCFAnalyser(r_max=50, dr=1, step=1)

analysis = pcf_collector.collect(experiment, str(analyser))
display(analysis.head(6))

Tumour-Vessel


100%|██████████| 21/21 [00:00<00:00, 774.05it/s]


r                    0    1    2    3    4    5    6    7    8    9   ...  \
iteration timestep                                                    ...   
1520      0         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
          3000      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
          6000      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
          9000      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
          12000     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
          15000     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

r                         41        42        43        44        45  \
iteration timestep                                                     
1520      0         1.127958  1.514450  1.883172  1.446385  1.414596   
          3000      1.274175  1.448161  1.634075  1.636314  1.352678   
          6000      1.459711  1.468558  1.614147  1.546936  1.391902   
          9000      1.509474  1.511751  1.487548  1.505684  1.447377   
          12000     1.517373  1.463313  1.457857  1.535297  1.462362   
          15000     1.501297  1.494413  1.462584  1.454410  1.422530   

r                         46        47        48        49        50  
iteration timestep                                                    
1520      0         1.174451  1.437158  1.286880  1.159742  1.013340  
          3000      1.304946  1.386971  1.253876  1.179617  1.041966  
          6000      1.292883  1.362348  1.256636  1.138080  1.121910  
          9000      1.361969  1.294651  1.271324  1.165209  1.081952  
          12000     1.354344  1.315956  1.238068  1.162626  1.166564  
          15000     1.393908  1.298857  1.260365  1.198962  1.150237  

[6 rows x 51 columns]

In [30]:
print("Tumour-Tumour")
analyser = cmm.analysers.TumourTumourPCFAnalyser(r_max=50, dr=1, step=1)

analysis = pcf_collector.collect(experiment, str(analyser))
display(analysis.head(6))

Tumour-Tumour


100%|██████████| 21/21 [00:00<00:00, 452.60it/s]


r                            0           1           2           3   \
iteration timestep                                                    
1520      0         2387.324146    0.000000    0.000000    0.000000   
          3000      1100.330471  445.371857   78.595034    0.000000   
          6000       616.794243  396.510585  189.443946   34.616003   
          9000       363.468036  312.068516  191.646783   85.753281   
          12000      214.371964  205.710471  151.229677  103.381112   
          15000      129.445599  132.181848  112.207234   87.409607   

r                          4          5          6         7         8   \
iteration timestep                                                        
1520      0          0.000000   0.000000   0.000000  0.000000  0.000000   
          3000       0.000000   0.000000   0.000000  0.000000  0.000000   
          6000       0.000000   0.000000   0.000000  0.000000  0.000000   
          9000      25.699760   0.000000   0.000000  0.000000  0.000000   
          12000     55.722275  23.622255   4.897075  0.086615  0.000000   
          15000     61.249869  38.948487  21.541885  9.955735  2.878632   

r                         9   ...   41   42   43   44   45   46   47   48  \
iteration timestep            ...                                           
1520      0         0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
          3000      0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
          6000      0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
          9000      0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
          12000     0.000000  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
          15000     0.199403  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

r                    49   50  
iteration timestep            
1520      0         0.0  0.0  
          3000      0.0  0.0  
          6000      0.0  0.0  
          9000      0.0  0.0  
          12000     0.0  0.0  
          15000     0.0  0.0  

[6 rows x 51 columns]

In [31]:
print("TCell-TCell")
analyser = cmm.analysers.TCellTCellPCFAnalyser(r_max=50, dr=1, step=1)

analysis = pcf_collector.collect(experiment, str(analyser))
display(analysis.head(6))

TCell-TCell


100%|██████████| 17/17 [00:00<00:00, 1035.36it/s]


r                         0         1         2         3         4   \
iteration timestep                                                     
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000   
          15000     0.000000  0.000000  0.000000  0.000000  0.000000   
          18000     0.000000  0.000000  0.000000  0.000000  0.000000   
          21000     0.000000  1.835697  1.101418  0.786727  1.223798   
          24000     4.527074  2.640793  3.168952  1.616812  1.886281   
          27000     1.096108  2.557586  2.630660  1.996670  1.509620   

r                         5         6         7         8         9   ...  \
iteration timestep                                                    ...   
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          15000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          18000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          21000     1.501934  0.423622  2.202837  1.943679  1.450222  ...   
          24000     1.569766  1.226692  1.509025  1.276617  1.256446  ...   
          27000     1.515635  1.399361  1.075218  0.997942  1.049745  ...   

r                         41        42        43        44        45  \
iteration timestep                                                     
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000   
          15000     0.000000  0.000000  0.000000  0.000000  0.000000   
          18000     1.735962  0.000000  1.900678  1.799538  0.000000   
          21000     1.436129  0.553326  1.753984  0.903194  0.775011   
          24000     0.785498  0.710095  1.132446  0.997886  0.953210   
          27000     1.198741  1.040531  1.175300  1.030512  0.962936   

r                         46        47        48        49        50  
iteration timestep                                                    
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000  
          15000     0.000000  0.000000  0.000000  0.000000  0.000000  
          18000     1.852262  0.000000  0.000000  0.000000  0.000000  
          21000     1.142834  0.646774  0.777296  0.878630  0.728087  
          24000     1.184811  1.153746  0.957064  1.029339  0.843798  
          27000     0.790380  1.096593  0.929793  1.024125  0.954041  

[6 rows x 51 columns]

In [32]:
print("Vessel-TCell")
analyser = cmm.analysers.VesselTCellPCFAnalyser(r_max=50, dr=1, step=1)

analysis = pcf_collector.collect(experiment, str(analyser))
display(analysis.head(6))

Vessel-TCell


100%|██████████| 17/17 [00:00<00:00, 752.32it/s]


r                         0         1         2         3         4   \
iteration timestep                                                     
1520      12000     0.000000  0.000000  4.681028  0.000000  0.866857   
          15000     0.000000  0.000000  2.080457  0.000000  1.155809   
          18000     1.733714  1.733714  2.165626  1.486041  0.963174   
          21000     0.458924  0.611899  0.826064  1.704576  1.395369   
          24000     0.208046  1.480770  1.414711  1.277995  0.950421   
          27000     0.591039  1.170460  1.040228  1.127496  1.118259   

r                         5         6         7         8         9   ...  \
iteration timestep                                                    ...   
1520      12000     0.709247  0.600132  1.040228  0.917849  0.410616  ...   
          15000     0.472831  0.000000  0.693486  1.415466  0.547489  ...   
          18000     1.576104  0.800176  0.809067  0.623754  0.758191  ...   
          21000     1.126451  1.210225  0.836260  0.894501  1.293383  ...   
          24000     1.051875  1.258158  1.042486  1.028735  1.111229  ...   
          27000     1.234851  1.027363  0.910466  0.980811  0.891210  ...   

r                         41        42        43        44        45  \
iteration timestep                                                     
1520      12000     1.008618  0.912331  1.561738  1.379216  0.914064   
          15000     1.589327  1.733361  1.451707  1.107048  1.184902   
          18000     1.052440  0.608518  1.392986  0.852249  0.940674   
          21000     0.989062  1.047045  1.052009  0.952375  0.914846   
          24000     1.016386  1.033412  0.935259  0.965412  1.005927   
          27000     1.203335  1.139249  1.106985  1.150756  1.186756   

r                         46        47        48        49        50  
iteration timestep                                                    
1520      12000     1.522722  0.384270  1.960407  0.352935  1.565453  
          15000     1.470643  1.583103  1.091543  1.023769  0.915444  
          18000     0.741472  0.908597  1.151017  0.833709  1.172065  
          21000     0.910380  0.947614  1.134014  1.057077  0.948443  
          24000     1.002077  1.041913  0.973405  0.974638  1.237958  
          27000     1.129815  1.147610  1.198162  1.175072  1.094205  

[6 rows x 51 columns]

In [33]:
print("Tumour-TCell")
analyser = cmm.analysers.TumourTCellPCFAnalyser(r_max=50, dr=1, step=1)

analysis = pcf_collector.collect(experiment, str(analyser))
display(analysis.head(6))

Tumour-TCell


100%|██████████| 17/17 [00:00<00:00, 689.67it/s]


r                         0         1         2         3         4   \
iteration timestep                                                     
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000   
          15000     0.000000  0.000000  0.000000  0.000000  0.000000   
          18000     0.000000  0.000000  0.000000  0.000000  0.000000   
          21000     0.000000  0.000000  0.000000  0.000000  0.000000   
          24000     1.300741  1.238801  1.323039  1.242340  1.214025   
          27000     2.979609  2.926054  2.915343  2.846069  2.736177   

r                         5         6         7         8         9   ...  \
iteration timestep                                                    ...   
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          15000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          18000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          21000     0.000000  0.000000  0.000000  0.000000  0.000000  ...   
          24000     1.148706  1.052028  0.921668  0.819795  0.631788  ...   
          27000     2.654295  2.515590  2.297999  2.106690  1.849572  ...   

r                         41        42        43        44        45  \
iteration timestep                                                     
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000   
          15000     0.000000  0.175813  0.687086  1.259336  1.313905   
          18000     1.453315  1.281341  1.036693  0.892620  0.752437   
          21000     1.511796  1.407778  1.286227  1.146293  1.014285   
          24000     1.275351  1.154724  1.057797  0.977622  0.877628   
          27000     1.104902  1.036973  0.983946  0.942235  0.895066   

r                         46        47        48        49        50  
iteration timestep                                                    
1520      12000     0.000000  0.000000  0.000000  0.000000  0.000000  
          15000     2.026249  1.762239  1.444025  1.498285  1.122934  
          18000     0.732530  0.743538  0.862281  0.843321  0.776188  
          21000     0.875803  0.739028  0.588019  0.397859  0.280191  
          24000     0.781861  0.712462  0.618636  0.558962  0.507218  
          27000     0.864278  0.847520  0.816774  0.785518  0.757473  

[6 rows x 51 columns]